In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from pandas_profiling import ProfileReport

warnings.filterwarnings("ignore")
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

C:\Users\ppizarro\AppData\Local\Temp\ipykernel_10844\1857573116.py:10: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


In [3]:
df = pd.read_csv("../data/interim/datos_limpios.csv", encoding="latin-1", sep=";")

## Analisis exporatorio previo al preprocesamiento

In [31]:
report = ProfileReport(df)
report.to_file("../reports/1.0-jrb-exploratory-data-analysis.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

## Analisis de Columna Codigo Diagnostico

- Esta columna carece de valores faltantes

In [37]:
largo_codigos = df["Código Diagnóstico"].str.len().value_counts(normalize=True, dropna=False)
display(largo_codigos * 100)

5    58.598147
3    27.038000
7     9.194450
2     1.983357
6     1.550353
4     1.053433
1     0.581900
8     0.000360
Name: Código Diagnóstico, dtype: float64

- El 58.5% de los codigos presenta largo 5. Los codigos con largo 5 son del tipo: "C38.4", por
lo que es el formato deseable

In [28]:
df[df["Código Diagnóstico"].str.len() == 5]["Código Diagnóstico"].sample(10)

493901    J95.5
525367    G47.3
252354    J84.9
518971    I50.0
513809    C34.3
25321     J44.9
290416    J44.9
331304    G47.3
171682    I35.0
379992    R01.1
Name: Código Diagnóstico, dtype: object


- El 27.0% de los codigos presenta largo 3. Los codigos con largo 3 son del tipo: "R05", "C34", por
lo que corresponde a la Categoria del diagnostico.


In [29]:
df[df["Código Diagnóstico"].str.len() == 3]["Código Diagnóstico"].sample(10)

12471     I25
236075    I99
481530    R91
270795    J84
144920    I99
439823    F17
438921    J47
233686    I99
92433     J47
23311     J84
Name: Código Diagnóstico, dtype: object


- El 9.20% de los codigos presenta largo 7. Los codigos con largo 7 son del tipo: "J44.8.4" o "E10-E16".


In [30]:
df[df["Código Diagnóstico"].str.len() == 7]["Código Diagnóstico"].sample(10)

243187    J45.9.1
300799    J44.8.3
465033    E65-E68
442316    J43.1.2
447507    J45.0.2
41828     J43.1.1
246421    J80-J84
477773    I20-I25
115291    J45.9.1
14223     I20-I25
Name: Código Diagnóstico, dtype: object


- El 1.98% de los codigos presenta largo 2. Los codigos con largo 2 son del tipo: "55", por lo que
tienen una notacion de diagnostico deficiente. Sin embargo, estos codigos se relacionan a un
unico "Nombre Diagnostico". Esto puede permitir recodificar estos codigos de forma mas facil.

In [31]:
df[df["Código Diagnóstico"].str.len() == 2][["Código Diagnóstico", "Nombre Diagnóstico"]].value_counts()

Código Diagnóstico  Nombre Diagnóstico                    
55                  Fibrosis pulmonar                         4335
79                  Trasplante  Pulmon                        2788
78                  Trasplante Cardiaco                       1197
41                  Nódulo pulmonar                            842
20                  Bronquiectasias                            363
54                  Enfermedad pulmonar difusa                 342
42                  Tumor pulmonar                             113
77                  Asma Bronquial en el Adulto (Sospecha)     110
74                  Tabaquismo                                 100
64                  Hipertensión pulmonar primaria              99
17                  Neumonitis por hipersensibilidad            98
49                  Disnea en estudio                           79
44                  Cáncer pulmonar                             53
21                  Bronquiectasias infectadas                  50
59 

- El 1.55% de los codigos presenta largo 6. Los codigos con largo 6 son del tipo: "300-23". Tambien tienen
una relacion 1:1 entre Código Diagnóstico y Nombre Diagnóstico.

In [35]:
df[df["Código Diagnóstico"].str.len() == 6][["Código Diagnóstico", "Nombre Diagnóstico"]].value_counts()

Código Diagnóstico  Nombre Diagnóstico                                                             
300-23              Tetralogía de Fallot (TOF)                                                         2523
300-32              Ventriculo Unico Funcional                                                         1385
300-28              Transposición Grandes Arterias (discordancia de la conexión ventriculoarterial)     848
300-29              Transposición Grandes Arterias + CIV                                                519
300-12              Doble Salida VD (DSVD)                                                              517
                                                                                                       ... 
902-12              Preexcitación ventricular                                                             2
I51.14              CIV Sub Aórtica (tipo fallot)                                                         2
902-27              Macroreentrada p

- El 1.05% de los codigos presenta largo 4 y son del tipo: "81-4" o "I32*"

In [36]:
df[df["Código Diagnóstico"].str.len() == 4][["Código Diagnóstico", "Nombre Diagnóstico"]].value_counts()

Código Diagnóstico  Nombre Diagnóstico                                     
81-4                Otro                                                       4553
88-1                Otro:                                                       300
81-6                Estenosis Pulmonar Valvular                                 254
91-3                Enfermedad Coronaria 3V                                     251
85-6                Cardiomiopatía Isquemica                                    126
91-1                Enfermedad Coronaria 1V                                      80
85-3                Enfermedad Coronaria (cardiopatía coronaria)                 73
91-2                Enfermedad Coronaria 2V                                      66
91-5                Aneurisma Aorta Ascendnte (sin mención de ruptura)           43
91-6                Aneurisma Raíz Aórtica (sin mención de ruptura)              25
81-5                Estenosis Pulmonar Subvalvular (Infundibular)                16


- El 0.58% de los codigos presenta largo 1, y son del tipo "5" o "6"

In [38]:
df[df["Código Diagnóstico"].str.len() == 1][["Código Diagnóstico", "Nombre Diagnóstico"]].value_counts()

Código Diagnóstico  Nombre Diagnóstico          
5                   Otro                            3072
6                   Ex TBC pulmonar                  107
2                   Crisis asmática                   23
7                   TBC pulmonar                      23
8                   TBC pulmonar multiresistente       5
9                   Micobacteriosis atipica            2
dtype: int64

- El 0.00036% de los codigos presenta largo 8, y son del tipo "Z00- Z13"

In [39]:
df[df["Código Diagnóstico"].str.len() == 8][["Código Diagnóstico", "Nombre Diagnóstico"]].value_counts()

Código Diagnóstico  Nombre Diagnóstico                                  
Z00- Z13            PRUEBAS PARA ACLARAR O INVESTIGAR PROBLEMAS DE SALUD    2
dtype: int64